#### This notebook is about analysing how models perform using different sets of features and machine learning techniques after obtaining the optimal hyperparameters settings for the machine learning techniques. The target is smoothed. (Currently set to 5 days average)

In [1]:
%matplotlib inline
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import warnings
from collections import Counter
from sklearn import neighbors, svm, model_selection, metrics, linear_model
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_validate, train_test_split
from sklearn.utils import resample
from sklearn import preprocessing
from tqdm import tqdm
import datetime as dt
import mpld3
from collections import Counter

mpld3.enable_notebook()
from statistical_metrics import *
from technical_indicators import *

In [2]:
pd.set_option("display.max_columns", 15)
pd.set_option("display.width", 500)

warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [3]:
def classification(df, n):
    df['Next_change'] = df['Change'].shift(-n)
    df.dropna(inplace=True)
    df['Target'] = np.where(df['Next_change'] > 0.0, 1, -1)
    df.drop(columns='Next_change', inplace=True)

    return df

def SMA_classification(df, n):
    df['Target ma'] = df['Price'].rolling(window=n, min_periods=n).mean()
    df['Target ma'] = df['Target ma'].round(4)
    df.dropna(inplace=True)
    df['Target'] = np.where(df['Target ma'.format(n)].shift(-n) > df['Target ma'], 1, -1)
    df.drop(columns=['Target ma'], inplace=True)
    df.dropna(inplace=True)

    return df

df = pd.read_csv('Data/EUR_USD_1418.csv', index_col=0)

look_ahead = 10
# Create labels for training
df = SMA_classification(df, look_ahead)

df.dropna(inplace=True)

df.drop(columns=['Change'], inplace=True)

df = moving_average(df, 5)
df = moving_average(df, 8)
df = moving_average(df, 10)
df = moving_average(df, 13)
df = moving_average(df, 15)
df = moving_average(df, 20)
df = moving_average(df, 21)
df = EMA(df, 5)
df = EMA(df, 10)
df = EMA(df, 15)
df = EMA(df, 20)
df = MACD(df, 12, 26, 9)
df = MACD(df, 21, 55, 13)
df = stochastic_oscillator(df, 5)
df = stochastic_oscillator(df, 8)
df = stochastic_oscillator(df, 10)
df = stochastic_oscillator(df, 13)
df = stochastic_oscillator(df, 15)
df = stochastic_oscillator(df, 20)
df = stochastic_oscillator(df, 21)
df = williams_R(df, 5)
df = williams_R(df, 7)
df = williams_R(df, 8)
df = williams_R(df, 10)
df = williams_R(df, 13)
df = williams_R(df, 14)
df = williams_R(df, 15)
df = williams_R(df, 20)
df = williams_R(df, 21)
df = relative_strength_index(df, 5)
df = relative_strength_index(df, 7)
df = relative_strength_index(df, 8)
df = relative_strength_index(df, 10)
df = relative_strength_index(df, 13)
df = relative_strength_index(df, 14)
df = relative_strength_index(df, 15)
df = relative_strength_index(df, 20)
df = relative_strength_index(df, 21)
df = momentum(df, 5)
df = momentum(df, 7)
df = momentum(df, 8)
df = momentum(df, 10)
df = momentum(df, 13)
df = momentum(df, 14)
df = momentum(df, 15)
df = momentum(df, 20)
df = momentum(df, 21)
df = change_rate(df, 1)
df = change_rate(df, 2)
df = change_rate(df, 3)
df = change_rate(df, 4)
df = change_rate(df, 5)
df = average_true_range(df, 7)
df = average_true_range(df, 14)
df = average_true_range(df, 21)
df = average_directional_index_and_DI(df, 5)
df = average_directional_index_and_DI(df, 7)
df = average_directional_index_and_DI(df, 8)
df = average_directional_index_and_DI(df, 10)
df = average_directional_index_and_DI(df, 13)
df = average_directional_index_and_DI(df, 14)
df = average_directional_index_and_DI(df, 15)
df = average_directional_index_and_DI(df, 20)
df = average_directional_index_and_DI(df, 21)
df.dropna(inplace=True)

print('Classification number:', Counter(df['Target']))
target_counts = dict(Counter(df['Target']).items())
print('Downtrend:', round(target_counts.get(-1)/df['Target'].shape[0], 3))
print('Uptrend:', round(target_counts.get(1)/df['Target'].shape[0], 3))

Classification number: Counter({-1: 680, 1: 544})
Downtrend: 0.556
Uptrend: 0.444


In [4]:
# Feature sets

def feature_set1(df):
    new_df = df[['Low', 'High', 'ema5', 'Open', '5ma', 'ema10', '8ma', '10ma', 'ema15', '13ma', '15ma', 'ema20', 
                 '20ma', '21ma', 'Target']].copy()
    
    return new_df

def feature_set2(df):
    new_df = df[['Price', 'Low', 'High', '21ma', '-DI 21', '5ma', '8ma', '10ma', 'ema5', 'momentum 10', 'momentum 14', 
                 'ADX 8', '-DI 15', '13ma', 'change_rate 2', 'Target']].copy()
        
    return new_df

def feature_set3(df):
    new_df = df[['Price', 'ema5', 'ema20', '21ma', 'ema15', 'ema10', '20ma', 'Low', 'High', 'Target']].copy()
        
    return new_df

switch = {
    'feature_set1': feature_set1,
    'feature_set2': feature_set2,
    'feature_set3': feature_set3
}

In [5]:
def feature_selector(df, feature_set):
    
    df = switch[feature_set](df)
    df.dropna(inplace=True)
    return df

In [6]:
def data_processing(df, feature_set):
    df = feature_selector(df, feature_set)
    
    df = df.replace([np.inf, -np.inf], np.nan)
    # Drop na, nan and nat rows
    df.dropna(inplace=True)
    
    tmp_df = df.copy()
    tmp_df.drop(columns=['Target'], inplace=True)
    
    X = tmp_df.values
    y = df['Target'].values

    return X, y, df

In [7]:
def build_model(clf, X_train, y_train, X_test, y_test):
    
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    prediction = clf.predict(X_test)
    
    return accuracy, prediction


In [8]:
def feature_comparison(original_df, split_pct, feature_list):
    
    clf1 = neighbors.KNeighborsClassifier()
    clf2 = svm.SVC(kernel='rbf', C=10)
    clf3 = svm.SVC(kernel='linear', C=10)
    clf4 = RandomForestClassifier(random_state=0)
    clf5 = VotingClassifier([('lsvc', svm.LinearSVC(C=10)),
                            ('knn', neighbors.KNeighborsClassifier()),
                            ('rfor', RandomForestClassifier(random_state=0))])
    
    clf_list = [clf1, clf2, clf3, clf4, clf5]
    clf_name_list = ['KNN', 'SVM.SVC.RBF', 'SVM.SVC.Linear', 'RandomForestClassifier', 'VotingClassifier']
    
    accuracy_list = []
    
    for n in range(len(feature_list)):
        for i in range(len(clf_list)):

            df = original_df.copy()
            X, y, df = data_processing(df, feature_list[n])

            split_pct = split_pct
            train_size = int(X.shape[0] * split_pct)

            X_train, X_test = X[0:train_size], X[train_size:]
            y_train, y_test = y[0:train_size], y[train_size:]

            accuracy, prediction = build_model(clf_list[i], X_train, y_train, X_test, y_test)

            print('clf={} feature_set={}'.format(clf_name_list[i], feature_list[n]))
            print('Accuracy:', round(accuracy, 4))
            print("Confusion matrix: ")
            print('{}\n'.format(metrics.confusion_matrix(y_test, prediction)))

            accuracy_list.append([round(accuracy, 4), 
                                  'clf={}, feature_set={}'.format(clf_name_list[i], feature_list[n])])

    accuracy_list = list(reversed(sorted(accuracy_list)))
    print('Highest accuracy: {}, Lowest accuracy: {}'.format(accuracy_list[0][0], accuracy_list[-1][0]))
    selected_model = round(0.2 * len(accuracy_list))
    for i in range(selected_model):
        print('Name: {}, Accuracy: {}'.format(accuracy_list[i][1], accuracy_list[i][0]))

In [9]:
# feature_list = ['feature_set1', 'feature_set2', 'feature_set3', 'feature_set4', 'feature_set5', 'feature_set6', 
#                 'feature_set7', 'feature_set8', 'feature_set9', 'feature_set10']

feature_list = ['feature_set1', 'feature_set2', 'feature_set3']
feature_comparison(df, 0.8, feature_list)


clf=KNN feature_set=feature_set1
Accuracy: 0.5143
Confusion matrix: 
[[89 71]
 [48 37]]

clf=SVM.SVC.RBF feature_set=feature_set1
Accuracy: 0.6041
Confusion matrix: 
[[130  30]
 [ 67  18]]

clf=SVM.SVC.Linear feature_set=feature_set1
Accuracy: 0.6245
Confusion matrix: 
[[106  54]
 [ 38  47]]

clf=RandomForestClassifier feature_set=feature_set1
Accuracy: 0.5224
Confusion matrix: 
[[92 68]
 [49 36]]

clf=VotingClassifier feature_set=feature_set1
Accuracy: 0.5429
Confusion matrix: 
[[94 66]
 [46 39]]

clf=KNN feature_set=feature_set2
Accuracy: 0.551
Confusion matrix: 
[[90 70]
 [40 45]]

clf=SVM.SVC.RBF feature_set=feature_set2
Accuracy: 0.6245
Confusion matrix: 
[[123  37]
 [ 55  30]]

clf=SVM.SVC.Linear feature_set=feature_set2
Accuracy: 0.649
Confusion matrix: 
[[123  37]
 [ 49  36]]

clf=RandomForestClassifier feature_set=feature_set2
Accuracy: 0.6531
Confusion matrix: 
[[120  40]
 [ 45  40]]

clf=VotingClassifier feature_set=feature_set2
Accuracy: 0.6449
Confusion matrix: 
[[117  43]

In [10]:
# Analysis
# window=5, 
# Highest accuracy: 0.6829, Lowest accuracy: 0.5
# Name: clf=SVM.SVC.RBF, feature_set=feature_set8, Accuracy: 0.6829
# Name: clf=SVM.SVC.RBF, feature_set=feature_set10, Accuracy: 0.6789
# Name: clf=SVM.SVC.Linear, feature_set=feature_set5, Accuracy: 0.6789
# Name: clf=SVM.SVC.Linear, feature_set=feature_set3, Accuracy: 0.6789
# Name: clf=SVM.SVC.Linear, feature_set=feature_set10, Accuracy: 0.6789
# Name: clf=VotingClassifier, feature_set=feature_set5, Accuracy: 0.6748
# Name: clf=VotingClassifier, feature_set=feature_set10, Accuracy: 0.6748
# Name: clf=SVM.SVC.Linear, feature_set=feature_set9, Accuracy: 0.6748
# Name: clf=SVM.SVC.Linear, feature_set=feature_set8, Accuracy: 0.6748
# Name: clf=SVM.SVC.Linear, feature_set=feature_set4, Accuracy: 0.6748

# window=10
# Highest accuracy: 0.702, Lowest accuracy: 0.4857
# Name: clf=VotingClassifier, feature_set=feature_set10, Accuracy: 0.702
# Name: clf=RandomForestClassifier, feature_set=feature_set6, Accuracy: 0.698
# Name: clf=VotingClassifier, feature_set=feature_set6, Accuracy: 0.6939
# Name: clf=RandomForestClassifier, feature_set=feature_set10, Accuracy: 0.6939
# Name: clf=SVM.SVC.Linear, feature_set=feature_set10, Accuracy: 0.6898
# Name: clf=RandomForestClassifier, feature_set=feature_set8, Accuracy: 0.6898
# Name: clf=SVM.SVC.Linear, feature_set=feature_set6, Accuracy: 0.6857
# Name: clf=SVM.SVC.RBF, feature_set=feature_set6, Accuracy: 0.6816
# Name: clf=SVM.SVC.RBF, feature_set=feature_set10, Accuracy: 0.6776
# Name: clf=SVM.SVC.Linear, feature_set=feature_set8, Accuracy: 0.6776

# window=15
# Highest accuracy: 0.7541, Lowest accuracy: 0.5082
# Name: clf=VotingClassifier, feature_set=feature_set10, Accuracy: 0.7541
# Name: clf=SVM.SVC.Linear, feature_set=feature_set1, Accuracy: 0.7541
# Name: clf=SVM.SVC.Linear, feature_set=feature_set2, Accuracy: 0.75
# Name: clf=RandomForestClassifier, feature_set=feature_set10, Accuracy: 0.7377
# Name: clf=SVM.SVC.RBF, feature_set=feature_set7, Accuracy: 0.7336
# Name: clf=SVM.SVC.Linear, feature_set=feature_set7, Accuracy: 0.7336
# Name: clf=RandomForestClassifier, feature_set=feature_set8, Accuracy: 0.7213
# Name: clf=SVM.SVC.Linear, feature_set=feature_set8, Accuracy: 0.7172
# Name: clf=SVM.SVC.RBF, feature_set=feature_set8, Accuracy: 0.709
# Name: clf=SVM.SVC.RBF, feature_set=feature_set6, Accuracy: 0.7049